In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import filtre
import altair as alt


np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.float_format", lambda x: "{:.3f}".format(x))
pd.set_option("display.max_columns", 10)
pd.set_option("display.max_rows", 20)
pd.set_option("display.min_rows", 10)

# (11.7, 8.27) = A4 landscape
sns.set_theme(style="dark", palette="muted", font_scale=1.10, rc={"figure.figsize": (16.54, 11.7)})


DATASET_FILENAME = Path("results/activities_2022-01-29_16-33-05.csv")
dataset = pd.read_csv(DATASET_FILENAME, index_col=[0, 1, 2], header=[0, 1, 2])

all_compounds = set(dataset.index.get_level_values(1))
all_activities = set(dataset.columns.get_level_values(1))
nb_acti = len(all_activities)
nb_comp = len(all_compounds)


sort_order = [True, True, False]
dataset.sort_index(axis=1, inplace=True, ascending=sort_order)
dataset.sort_index(axis=0, inplace=True, ascending=sort_order)


assert nb_comp * 2 == len(dataset.index)
assert nb_acti * 2 == len(dataset.columns)
dataset

filtre.keep_with_or_without(dataset, "w/")

abiotic          \
                                                      antioxidant drought   
alkaloid          acridine                                    266       2   
                  benzylamine                                  71       0   
                  colchicine                                  192       4   
                  cyclopeptide                                 57       1   
                  imidazole                                  1082       8   
...                                                           ...     ...   
terpenoid/terpene polyterpene                                   0       0   
                  sesquiterpene                               863      17   
                  sesterterpene                                 7       0   
                  tetraterpene/carotenoid/xanthophyll        7587     345   
                  triterpene                                 1157       3   

                                                                        ...  \
                                                      metal  salt   uv  ...   
alkaloid          acridine                              257   163   80  ...   
                  benzylamine                           165    80   23  ...   
                  colchicine                             84   187   20  ...   
                  cyclopeptide                          168    35   16  ...   
                  imidazole                            2507  1195  302  ...   
...                                                     ...   ...  ...  ...   
terpenoid/terpene polyterpene                             0     0    0  ...   
                  sesquiterpene                          72    59   39  ...   
                  sesterterpene                           0     3    0  ...   
                  tetraterpene/carotenoid/xanthophyll   592   513  392  ...   
                  triterpene                             51    59   46  ...   

                                                      pharmaco             \
                                                       obesity rheumatism   
alkaloid          acridine                                   3          0   
                  benzylamine                               34          0   
                  colchicine                               172         19   
                  cyclopeptide                              45          7   
                  imidazole                                297          1   
...                                                        ...        ...   
terpenoid/terpene polyterpene                                0          0   
                  sesquiterpene                             49          0   
                  sesterterpene                              1          0   
                  tetraterpene/carotenoid/xanthophyll      418          1   
                  triterpene                               127          2   

                                                                     toxicity  
                                                      sedative wound toxicity  
alkaloid          acridine                                   8    77      402  
                  benzylamine                                6    19      102  
                  colchicine                                34   222      866  
                  cyclopeptide                               4    51      171  
                  imidazole                                486   460     1768  
...                                                        ...   ...      ...  
terpenoid/terpene polyterpene                                0     0        2  
                  sesquiterpene                             30   124      341  
                  sesterterpene                              1     2       12  
                  tetraterpene/carotenoid/xanthophyll        3    97      828  
                  triterpene                                22   2

In [2]:
testGraph = filtre.keep_with_or_without(dataset, "w/")

comp_class = list(testGraph.T.index.get_level_values(0))
comp = list(testGraph.index.get_level_values(1))

acti_class = list(testGraph.T.index.get_level_values(0))
acti = [list(testGraph.T.index.get_level_values(1))]*53


source = pd.DataFrame({
    "value": ([item for sublist in testGraph.values for item in sublist]),
    "acti": [f"{acti[i][y]}" for i in range(len(acti)) for y in range(len(acti[i]))],
    "com": [item for sublist in [comp]*len(testGraph.T) for item in sublist],
    "class": [item for sublist in [comp_class]*len(testGraph) for item in sublist],
})



source

,value,acti,com,class
0,266,antioxidant,acridine,abiotic
1,2,drought,benzylamine,abiotic
2,257,metal,colchicine,abiotic
3,163,salt,cyclopeptide,abiotic
4,80,uv,imidazole,abiotic
...,...,...,...,...
1744,127,obesity,polyterpene,pharmaco
1745,2,rheumatism,sesquiterpene,pharmaco
1746,22,sedative,sesterterpene,pharmaco
1747,203,wound,tetraterpene/carotenoid/xanthophyll,pharmaco


In [3]:
testGraph = filtre.keep_with_or_without(dataset, "w/")
comp = list(testGraph.index.get_level_values(1))


In [5]:
min_value = 0
max_value = 1000



slider = alt.binding_range(min=min_value, max=max_value, step=1, name="papers threshold ")
select_slider = alt.selection_single(name="slider", fields=['cutoff'],
                                   bind=slider, init={'cutoff': 0})


input_dropdown = alt.binding_select(options=list(comp), name='selector of compound ')
selection = alt.selection_single(name="selection", fields=['com'], bind=input_dropdown, init={'com': list(comp)[0]})

vbar_test = alt.Chart(source).mark_bar().encode(
    x='acti:N',
    # use :Q to force the column to be sort 
    y='value:Q',
    tooltip=['acti', 'value', "class"],
    color = alt.Color('class', scale = alt.Scale(domain=comp_class))
).properties(
    height= 700,
    width = 900
).add_selection(selection, select_slider).transform_filter(
    'selection.com == datum.com && datum.value >= slider.cutoff[0]'
)


vbar_test.save('json_chart/vbar.json')
vbar_test

alt.Chart(...)